-calculate enters and exits for each 4-hour block

Look at:
-turnstile volume
-can we filter out student swipes?
-only look at spring for past couple of years (maybe compare to entire years)
-exits in evening: income (where they live); enters in evening: employers (work)

For presentation:
-top stations (top 10?) and best times

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.precision', 3)

In [2]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [150307, 150314, 150321, 150328, 160305, 160312, 160319, 160326,  170304, 170311, 170318, 170325]
ts = get_data(week_nums)

In [3]:
ts.columns = [column.strip() for column in ts.columns]
ts = ts[ts.DESC == 'REGULAR']

In [4]:
ts = ts.drop(["DESC"], axis=1, errors="ignore")

In [6]:
ts['DATETIME'] = pd.to_datetime(ts['DATE'] + ' ' + ts['TIME'], format="%m/%d/%Y %H:%M:%S")+ pd.Timedelta(minutes = 30)
ts['DATETIME'] = ts['DATETIME'].apply(lambda L: pd.datetime(L.year, L.month, L.day, L.hour))

In [7]:
ts.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)
#Should not be a big deal. Number of times we round up is low and difference in entries / exits is also probably low.

In [8]:
#Need to check that next row is 4 hours later. shift "Next DATETIME". Check occurences of it. Doesn't matter if < 4.
#The worry is that there's a large difference in times, such as 5 AM -> 5 PM which would give a very large delta.
ts[["NEXT_EXITS", "NEXT_ENTRIES"]] = (ts.groupby(["C/A", "UNIT", "SCP", "STATION"])["EXITS", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(-1)))

In [9]:
ts.dropna(subset=["NEXT_EXITS"], axis=0, inplace=True)
ts['DELTA_ENTRIES'] = ts['NEXT_ENTRIES']-ts['ENTRIES']
ts['DELTA_EXITS'] = ts['NEXT_EXITS']-ts['EXITS']

In [11]:
def get_delta_counts(row, max_counter,higher,lower):
    counter = row[higher] - row[lower]
    if counter < 0:
        # May be counter is reversed?
        counter = -counter
    if counter > max_counter:
        #print(row[higher], row[lower])
        #Looks like a reset
        counter = min(row[higher], row[lower])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

ts["DELTA_ENTRIES"] = ts.apply(get_delta_counts, axis=1, max_counter=10000,higher = 'NEXT_ENTRIES',lower = 'ENTRIES')
ts["DELTA_EXITS"] = ts.apply(get_delta_counts, axis=1, max_counter=10000,higher = 'NEXT_EXITS',lower = 'EXITS')

In [13]:
ts['DOTW']= ts['DATETIME'].dt.dayofweek
ts['IS_WEEKDAY']=True
ts.loc[(ts['DOTW'] == 5) | (ts['DOTW']==6),'IS_WEEKDAY'] = False

In [14]:
ts['TIMET'] = pd.to_datetime(ts['TIME'])+ pd.Timedelta(minutes = 30)
ts['TIMET'] = ts['TIMET'].apply(lambda L: pd.datetime(L.year, L.month, L.day, L.hour))

In [15]:
ts['HOUR']=pd.DatetimeIndex(ts['DATETIME']).hour

In [20]:
ts.loc[(ts['HOUR'] < 12) & (ts['HOUR']>= 4),'TOD'] = 'MORNING'
ts.loc[(12<=ts['HOUR']) & (ts['HOUR'] < 20),'TOD'] = 'EVENING'
ts.dropna(subset=["TOD"], axis=0, inplace=True)

In [49]:
entries = ts.groupby(['STATION','LINENAME','TOD','IS_WEEKDAY','DATE'])['DELTA_ENTRIES'].sum().reset_index().groupby(['STATION','LINENAME','TOD','IS_WEEKDAY']).mean()
ent_df = entries.reset_index()
exits = ts.groupby(['STATION','LINENAME','TOD','IS_WEEKDAY','DATE'])['DELTA_EXITS'].sum().reset_index().groupby(['STATION','LINENAME','TOD','IS_WEEKDAY']).mean()
exi_df = exits.reset_index()

In [82]:
ent_top_eve_wd=ent_df[(ent_df['TOD']=='EVENING') & (ent_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_ENTRIES', ascending=False).head(40)
ent_top_mor_wd=ent_df[(ent_df['TOD']=='MORNING') & (ent_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_ENTRIES', ascending=False).head(40)
ent_top_mor_we=ent_df[(ent_df['TOD']=='MORNING') & (ent_df['IS_WEEKDAY'] == 0)].sort_values(by = 'DELTA_ENTRIES', ascending=False).head(40)
ent_top_eve_we=ent_df[(ent_df['TOD']=='EVENING') & (ent_df['IS_WEEKDAY'] == 0)].sort_values(by = 'DELTA_ENTRIES', ascending=False).head(40)
exi_top_eve_wd=exi_df[(exi_df['TOD']=='EVENING') & (exi_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_EXITS', ascending=False).head(40)
exi_top_mor_wd=exi_df[(exi_df['TOD']=='MORNING') & (exi_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_EXITS', ascending=False).head(40)
exi_top_eve_we=exi_df[(exi_df['TOD']=='EVENING') & (exi_df['IS_WEEKDAY']==0)].sort_values(by = 'DELTA_EXITS', ascending=False).head(40)
exi_top_mor_we=exi_df[(exi_df['TOD']=='MORNING') & (exi_df['IS_WEEKDAY']==0)].sort_values(by = 'DELTA_EXITS', ascending=False).head(40)

In [86]:
ent_top_eve_wd.to_csv('entries_evening_weekday.csv')
ent_top_mor_wd.to_csv('entries_morning_weekday.csv')
ent_top_eve_we.to_csv('entries_evening_weekend.csv')
ent_top_mor_we.to_csv('entries_morning_weekend.csv')
exi_top_eve_wd.to_csv('exits_evening_weekday.csv')
exi_top_mor_wd.to_csv('exits_morning_weekday.csv')
exi_top_eve_we.to_csv('exits_evening_weekend.csv')
exi_top_mor_we.to_csv('exits_morning_weekend.csv')

#bus_day=pd.merge(ent_top_eve_wd,exi_top_mor_wd,on=['STATION','LINENAME'])

In [117]:
ent_eve_wd=ent_df[(ent_df['TOD']=='EVENING') & (ent_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_ENTRIES', ascending=False)
exi_mor_wd=exi_df[(exi_df['TOD']=='MORNING') & (exi_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_EXITS', ascending=False)
ent_mor_wd=ent_df[(ent_df['TOD']=='MORNING') & (ent_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_ENTRIES', ascending=False)
exi_eve_wd=exi_df[(exi_df['TOD']=='EVENING') & (exi_df['IS_WEEKDAY'])].sort_values(by = 'DELTA_EXITS', ascending=False)
liv_day=pd.merge(ent_mor_wd,exi_eve_wd,on=['STATION','LINENAME'])
liv_day['AVERAGE_DAILY']=bus_day['DELTA_ENTRIES']+bus_day['DELTA_EXITS']
liv_day_top = liv_day.sort_values(by = 'AVERAGE_DAILY', ascending=False).head(60)
liv_day_top.to_csv('liv_day_top.csv')

In [ ]:
bus_day=pd.merge(ent_eve_wd,exi_mor_wd,on=['STATION','LINENAME'])
bus_day['AVERAGE_DAILY']=bus_day['DELTA_ENTRIES']+bus_day['DELTA_EXITS']
bus_day_top = bus_day.sort_values(by = 'AVERAGE_DAILY', ascending=False).head(40)
bus_day_top.to_csv('bus_day_top.csv')
bus_day.to_csv('bus_day_daily.csv')

In [104]:
entries_t = test_df.groupby(['STATION','LINENAME','TOD','IS_WEEKDAY','DATE'])['DELTA_ENTRIES'].sum().reset_index().groupby(['STATION','LINENAME','TOD','IS_WEEKDAY']).mean()
ent_df_t = entries_t.reset_index()

In [ ]:
#Specifically look at 14th street station.
bus_day[bus_day['STATION'] == '14 ST'].sort_values(by = 'AVERAGE_DAILY', ascending=False)